In [ ]:
!pip install -qU \
  langchain-core  \
  langchain[google-genai]  \
  langchain-community  \
  langsmith  \
  google-search-results 

In [ ]:
import getpass
import os
from langchain_google_genai import GoogleGenerativeAI

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

#llm = GoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.0) 


from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [ ]:
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

# Define the multiply tool
@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

# Define the exponentiate tool
@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt= ChatPromptTemplate.from_messages([
    ("system", (
    "You're a helpful assistant. When answering a user's question "
        "you should first use one of the tools provided. After using a "
        "tool the tool output will be provided in the "
        "'scratchpad' below. If you have an answer in the "
        "scratchpad you should not use any more tools and "
        "instead answer directly to the user."
    )), 
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")    
])

In [ ]:
from langchain_core.runnables.base import RunnableSerializable

tools = [add, subtract, multiply, exponentiate]

agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")
)

In [ ]:
tool_call = agent.invoke({"input": "What is 10 + 10", "chat_history": []})

tool_call

In [ ]:
name2tool = {tool.name: tool.func for tool in tools}

name2tool

In [ ]:
tool_exec_content = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)

tool_exec_content

In [ ]:
from langchain_core.messages import ToolMessage

tool_exec = ToolMessage(
    content = f"The { tool_call.tool_calls[0]['name'] }  tool returned {tool_exec_content}",
    tool_call_id = tool_call.tool_calls[0]["id"]
)

out = agent.invoke(
    {
        "input": "What is 10 + 10",
        "chat_history": [],
        "agent_scratchpad": [tool_call, tool_exec]
    }
)

out

In [ ]:
agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="auto")
)

tool_call = agent.invoke({"input": "What is 10 + 10", "chat_history": []})
tool_call

In [ ]:
tool_output = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)

tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_output}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input": "What is 10 + 10",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_exec]
})
out

In [ ]:
@tool
def final_answer(answer: str, tools_used:list[str]) -> str:
    """Use this tool to provide a final answer to the user.
    The answer should be in natural language as this will be provided
    to the user directly. The tools_used must include a list of tool
    names that were used within the `scratchpad`.
    """
    return {"answer": answer, "tools_used": tools_used}
    

In [ ]:
tools = [final_answer, add, subtract, multiply, exponentiate]

name2tool = {tool.name: tool.func for tool in tools}

agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")  # we're forcing tool use again
)

In [ ]:
tool_call = agent.invoke({"input": "What is 10 + 10", "chat_history": []})
tool_call.tool_calls

In [ ]:
tool_out = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)

tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_out}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input": "What is 10 + 10",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_exec]
})
out

In [ ]:
out.tool_calls[0]["args"]

In [ ]:
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage

In [ ]:
import json

class CustomAgentExecutor:
    chat_history: list[BaseMessage]
    
    
    def __init__(self):
        self.chat_history = [] 
        
        self.agent: RunnableSerializable= (
            {
                "input": lambda x: x["input"],
                "chat_history": lambda x: x["chat_history"],
                "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])                
            }
            | prompt
            | llm.bind_tools(tools, tool_choice = "any")
        )
        
    
    def invoke(self, input: str, max_iterations:int = 3) -> dict:
        count = 0
        agent_scratchpad = []
        self.max_iterations = max_iterations
        
        while count < self.max_iterations:
            tool_call = self.agent.invoke({
                "input": input,
                "chat_history": self.chat_history,
                "agent_scratchpad": agent_scratchpad
            })
            
            agent_scratchpad.append(tool_call)
            
            tool_name = tool_call.tool_calls[0]["name"]
            tool_args = tool_call.tool_calls[0]["args"]
            tool_call_id = tool_call.tool_calls[0]["id"]
            tool_out = name2tool[tool_name](**tool_args)
            
            tool_exec = ToolMessage(
                content = tool_out,
                tool_call_id = tool_call_id
            )
            
            agent_scratchpad.append(tool_exec)
            print(f"{count}: {tool_name}({tool_args})")
            count += 1
            
            if tool_name == "final_answer":
                break
        
        if isinstance(tool_out, dict) and "answer" in tool_out:
            final_answer = tool_out["answer"]
            self.chat_history.extend([
                HumanMessage(content=input),
                AIMessage(content=final_answer)
            ])
            return json.dumps(tool_out)

        print("Need more iterations to get the answer")
        
        return json.dumps({})

In [ ]:
agent_executor_custom = CustomAgentExecutor()



In [ ]:
agent_executor_custom.invoke(input = "What is 10 + 10")

In [ ]:
agent_executor_custom.invoke(input = "What is nine plus 10, minus 4 * 2 to the power of 3", max_iterations = 5)